 Find top user(s) who have sent and received the most messages. Allow for ties and return answer ordered by total_messages desc

 ``` 
 table: sms_msgs
 schema: 	
message_id	bigint
sender_id	bigint
receiver_id	bigint
message_content_hashed	varchar
connection_quality	varchar
sent_at_date	timestamp
```

In [ ]:
with cte_combined_messages as (
select message_id, sender_id, receiver_id
from
sms_msgs
  
union all
  
select message_id, receiver_id as sender_id , sender_id as receiver_id
from
sms_msgs
  )
Select sender_id , 
count(message_id) as total_messages 

from
cte_combined_messages
group by 1
qualify dense_rank() over (order by count(message_id) desc) = 1


In [ ]:
### MAX Streak Length

In [ ]:
WITH login_dates AS (
  SELECT DISTINCT user_id, login_date
  FROM logins
),
grouped_logins AS (
  SELECT 
    user_id,
    login_date,
    DATEADD('day', -ROW_NUMBER() OVER (
      PARTITION BY user_id ORDER BY login_date
    ), login_date) AS streak_group
  FROM login_dates
),
streaks AS (
  SELECT 
    user_id,
    MIN(login_date) AS streak_start,
    MAX(login_date) AS streak_end,
    COUNT(*) AS streak_length
  FROM grouped_logins
  GROUP BY user_id, streak_group
),
ranked_streaks AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY user_id 
           ORDER BY streak_length DESC, streak_start
         ) AS rn
  FROM streaks
)
SELECT user_id, streak_start, streak_end, streak_length
FROM ranked_streaks
WHERE rn = 1;


### Conversion Funnel

Funnel Stages:

- Visited site
- Signed up
- Created first project
- Made a purchase

In [ ]:
#### conversion funnel

WITH funnel_raw AS (
  SELECT DISTINCT user_id,
    MAX(CASE WHEN event_name = 'visited_site' THEN 1 ELSE 0 END) AS visited,
    MAX(CASE WHEN event_name = 'signed_up' THEN 1 ELSE 0 END) AS signed_up,
    MAX(CASE WHEN event_name = 'created_project' THEN 1 ELSE 0 END) AS created_project,
    MAX(CASE WHEN event_name = 'made_purchase' THEN 1 ELSE 0 END) AS made_purchase
  FROM events
  GROUP BY user_id
),
funnel_counts AS (
  SELECT
    COUNT(DISTINCT user_id) AS visited_users,
    COUNT(DISTINCT CASE WHEN signed_up = 1 THEN user_id END) AS signed_up_users,
    COUNT(DISTINCT CASE WHEN created_project = 1 THEN user_id END) AS created_project_users,
    COUNT(DISTINCT CASE WHEN made_purchase = 1 THEN user_id END) AS purchase_users
  FROM funnel_raw
),
final_funnel AS (
  SELECT
    visited_users,
    signed_up_users,
    created_project_users,
    purchase_users,
    ROUND(100.0 * signed_up_users / visited_users, 2) AS pct_signed_up,
    ROUND(100.0 * created_project_users / signed_up_users, 2) AS pct_created_project,
    ROUND(100.0 * purchase_users / created_project_users, 2) AS pct_purchase
  FROM funnel_counts
)
SELECT * FROM final_funnel;


### cohort analysis
```
user_events (
  user_id STRING,
  event_name STRING,
  event_time TIMESTAMP
)

In [ ]:
WITH user_signup AS (
  -- Step 1: Get signup week per user 
  SELECT 
    user_id,
    MIN(DATE_TRUNC('WEEK', event_time)) AS signup_week
  FROM user_events
  WHERE event_name = 'signup'
  GROUP BY user_id
),
user_activity AS (
  -- Step 2: Get all activity weeks
  SELECT 
    e.user_id,
    u.signup_week,
    DATE_TRUNC('WEEK', e.event_time) AS activity_week
  FROM user_events e
  JOIN user_signup u ON e.user_id = u.user_id
),
weekly_cohorts AS (
  -- Step 3: Calculate week offset from signup
  SELECT 
    signup_week,
    DATEDIFF('WEEK', signup_week, activity_week) AS week_number,
    COUNT(DISTINCT user_id) AS active_users
  FROM user_activity
  GROUP BY signup_week, week_number
)
-- Step 4: Pivot or visualize in BI tool (like Tableau or Mode)
SELECT *
FROM weekly_cohorts
ORDER BY signup_week, week_number;


In [ ]:
### subscription overlap

SELECT s1.user_id, s1.subscription_id, s2.subscription_id
FROM subscriptions s1
JOIN subscriptions s2
  ON s1.user_id = s2.user_id
 AND s1.subscription_id < s2.subscription_id
 AND s1.start_date <= s2.end_date
 AND s2.start_date <= s1.end_date;

- Loss rate by month
- Loss rate within 30 days of transaction

In [ ]:
with cte_trans AS (
payment_id, date_trunc('month', transaction_date) as date_month ,
SUm(transaction_amount_usd) as total_trans
    from
    payment
    group by 
date_trunc('month', transaction_date), payment_id
),
cte_loss AS (
transaction_id, date_trunc('month', loss_event_date) as date_month ,
SUm(loss_charge_off_amt) as total_loss
    from
    loss
    group by 
date_trunc('month', loss_event_date), transaction_id
)
select
t.date_month,
total_loss*100.0/total_trans as loss_rate
from
cte_trans t
join cte_loss  l on on t.payment_id = l.transaction_id
group by t.date_month


In [ ]:
with cte_trans AS (
    select
    p.transaction_date,
    l.loss_event_date,
    p.transaction_amount,
    l.loss_amount
    from
    payment p
    left join loss l on p.payment_id = l.transaction_id and l.loss_event_date <= date_add('day',30,p.transaction_date)
)
select date_trunc('month',transaction_date) as date_month, sum(  transaction_amount) as total_trans, sum(loss_amount) as total_loss,
total_loss/total_trans as loss_rate
from
cte_trans
